# ner model 테스트

In [70]:
import pandas as pd
from tqdm import tqdm
import os, sys
from Read_Data import ReadKiprisData
from transformers import ElectraModel, ElectraTokenizer
from typing import Dict, Tuple
import re

In [2]:
# path 설정
patent_data_path = "./data/ner_dic_ami"
save_path = "./data/output"

In [59]:
# 1. 데이터 읽기
read_data = ReadKiprisData()
data_df = read_data.run(patent_data_path)
print(len(data_df))
data_df.head()

100%|██████████| 4/4 [00:00<00:00,  7.70it/s]

135843


,특허번호,문장번호,문장,단어,tag,BIO
0,1.020190e+12,1.0,"1. 분산처리 작업 데이터를 처리하는 단말기와,상기 분산처리 작업 데이터를 상기 단...",1,SN,O
1,1.020190e+12,1.0,"1. 분산처리 작업 데이터를 처리하는 단말기와,상기 분산처리 작업 데이터를 상기 단...",.,SF,O
2,1.020190e+12,1.0,"1. 분산처리 작업 데이터를 처리하는 단말기와,상기 분산처리 작업 데이터를 상기 단...",분산,NNG,B_TRM
3,1.020190e+12,1.0,"1. 분산처리 작업 데이터를 처리하는 단말기와,상기 분산처리 작업 데이터를 상기 단...",처리,NNG,I_TRM
4,1.020190e+12,1.0,"1. 분산처리 작업 데이터를 처리하는 단말기와,상기 분산처리 작업 데이터를 상기 단...",작업,NNG,I_TRM


In [4]:
len(data_df['특허번호'].unique())

129

In [5]:
len(data_df['문장'].unique()), len(data_df.groupby('문장'))

(2207, 2207)

In [6]:
# 2. input에 맞게 데이터 처리

# [entity:idx], [idx:entity] 딕셔너리 반환 
def lable_to_ner(data_df: pd.DataFrame, text: str='-') -> Tuple[Dict[int, str], Dict[str, int]]:
    label_group  = data_df.groupby('BIO')
    
    idx = 0
    label_to_idx = dict(); idx_to_label = dict()
    label_to_idx[text] = idx; idx_to_label[idx]=text # '[CLS]', '[SEP]'

    for label, _ in label_group:
        idx += 1
        idx_to_label[idx] = label
        label_to_idx[label] = idx

    return label_to_idx, idx_to_label

label_to_idx, idx_to_label = lable_to_ner(data_df, '-')

In [7]:
sen_group = data_df.groupby('문장')
sen_targets = []

for idx, group in tqdm(sen_group, desc="단어, BIO 태그"):
    sen_target = []

    sen_target.append(["CLS", "-"])
    for _, row in group.iterrows():
        if pd.notnull(row['단어']):
            sen_target.append([row["단어"], row["BIO"]])
    sen_target.append(["SEP", "-"])

    sen_targets.append(sen_target)

단어, BIO 태그: 100%|██████████| 2207/2207 [00:10<00:00, 202.17it/s]


In [8]:
sentences = []; targets = []; errors = []
for sen in sen_targets:
    sentence = []; target = []

    for word, bio in sen:
        sentence.append(word)
        try:
            target.append(label_to_idx[bio])
        except KeyError:
            target.append(label_to_idx['O'])
        except Exception as e:
            errors.append([word, bio, e])

    sentences.append(sentence); targets.append(target)

In [9]:
len(sentences), len(targets)

(2207, 2207)

In [10]:
# 최종 태깅된 bio 태그 건수
bio_count = dict()
flatten_target = [t for target in targets for t in target]
for target in flatten_target:
    if target not in bio_count: bio_count[target] = 0
    else: bio_count[target]+=1
bio_count

{0: 4413, 3: 98659, 1: 18717, 2: 18464}

In [11]:
word_count = 0
flatten_sentence = [s for sen in sentences for s in sen]
for sen in flatten_sentence:
    word_count += len(sen)
word_count

246015

In [13]:
# 3. 토크나이즈

# 모델 경로 (KIPI-KorPatELECTRA 경로 설정)
model_path = "../old/KIPI-KorPatELECTRA/KorPatELECTRA/PT"

# 토크나이저 및 모델 불러오기
tokenizer = ElectraTokenizer.from_pretrained(model_path)
model = ElectraModel.from_pretrained(model_path)

Some weights of the model checkpoint at ../old/KIPI-KorPatELECTRA/KorPatELECTRA/PT were not used when initializing ElectraModel: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
